In [1]:
!nvidia-smi

Wed Nov 25 06:43:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13021         572        7791           0        4657       12177
Swap:             0           0           0


## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [3]:
!pip install transformers

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

#phobert = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
custokenizer.add_tokens('\n')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


1

In [5]:
len(custokenizer)

64002

In [6]:
train_path = '68cleanregex.txt'
test_path = '68cleanvl_test.txt'

In [7]:
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time

logger = logging.get_logger(__name__)
class CusTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )

        # Make sure only the first process in distributed training processes the dataset,
        # and the others will use the cache.
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()

                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                i = 0
                while i < len(tokenized_text) - block_size + 1:
                    inds = tokenized_text[i : i + block_size]
                    # for j in range(0, len(inds)):
                    #     if inds[j]==64001:
                    #         inds = inds[j+1:] #remove the first \n
                    #         break
                    # for j in range(len(inds)-1, 0, -1):
                    #     if inds[j]==64001:
                    #         inds = inds[:j-1] #remove \n
                    #         break
                    i += len(inds)
                    self.examples.append(
                        tokenizer.build_inputs_with_special_tokens(inds)
                    )
                # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
                # If your dataset is small, first you should loook for a bigger one :-) and second you
                # can change this behavior by adding (model specific) padding.

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [8]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size=256)
     
    test_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

In [9]:
len(train_dataset), len(test_dataset)

(1672, 50)

In [10]:
print(custokenizer.decode(train_dataset[0]))

<s> đường thu vàng cả thời gian 
 gió se se lạnh vít làn thu sang 
 đường quê nhạt nắng hanh vàng 
 bao nhiêu kỷ niệm mơ màng trong ta 
 gió thu môi ấm mặn mà 
 ven hồ liễu rủ tóc nhoà điểm sương 
 trời xanh chiều tím quê hương 
 tìm về ký ức sắc hường ngẩn ngơ 
 chông chênh cây trút lá rồi 
 cành ươm mầm nhú khoảng trời nhấp nhô 
 hương quê trên đường ngoại ô 
 thu cùng ta đến bến bờ yêu thương 
 
 trăng là hoa của trời trong 
 sao kia tựa những cánh ong rụng rời 
 gió đàn hưu hắt người ơi 
 mây tơ con nhện rối bời không gian 
 trời cao mây gợn nắng lan 
 gió heo may thổi miên man bên rào 
 vài con bướm trắng lao xao 
 vàng bông hoa cúc cài vào vạt bay 
 thu rồi đồng rải heo may 
 bờ tre ngả úa sương bay là là 
 
 tìm về một thuở hạ xưa 
 bâng khuâng lối nhỏ rào thưa ngày nào 
 tìm vầng trăng rụng cầu ao 
 có con đom đóm đậu vào vai tôi 
 về tìm cái giậu mùng tơi 
 có cô hàng xóm thường chơi bên nhà 
 tìm về một khúc dân ca 
 có đàn cò trắng bay ra cánh đồng 
 tìm về nơi ấy dòng sông 

In [11]:
train_dataset[0].shape

torch.Size([256])

# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [12]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

Define model

In [13]:
pretrainedgpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
# pretrainedgpt2.resize_token_embeddings(len(custokenizer))
pretrainedgpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [14]:
task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [15]:
configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task,
                           eos_token_id=custokenizer.eos_token_id,
                           bos_token_id=custokenizer.bos_token_id, 
                           pad_token_id=custokenizer.pad_token_id,
                           sep_token_id=custokenizer.sep_token_id
                           )
poem = GPT2LMHeadModel(configuration)
poem

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

In [16]:
poem.config
# poem.resize_token_embeddings(len(custokenizer))
# poem.load_state_dict(pretrainedgpt2.state_dict())

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 258,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 258,
  "pad_token_id": 1,
  "resid_pdrop": 0.1,
  "sep_token_id": 2,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 256
    }
  },
  "vocab_size": 64002
}

In [17]:
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline
class PrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if int(state.epoch)%10 == 0:
            pipe = pipeline('text-generation', model=model, tokenizer=custokenizer, device=0)
            with open("sample.txt", "a") as f:
                f.write(pipe('<s> tìm về một thuở hạ xưa')[0]['generated_text'])
                f.write("\n===========================================\n")
                f.close()

In [18]:
training_args = TrainingArguments(
    output_dir="gpt2-vnmesepoem", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    save_steps=10000, # after # steps model is saved 
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=500,    # number of warmup steps for learning rate scheduler
    logging_dir='logs', # directory for storing logs
    logging_steps=500,
    )

In [19]:
trainer = Trainer(
    model=poem,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
    callbacks = [PrinterCallback],
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [20]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,7.760840
1000,6.128813
1500,5.631996
2000,5.258809
2500,4.963191
3000,4.671715
3500,4.405668
4000,4.130602
4500,3.842855
5000,3.565418


KeyboardInterrupt: ignored

In [22]:
from transformers import pipeline

poem = pipeline('text-generation', model="/content/gpt2-vnmesepoem/checkpoint-10000", tokenizer=custokenizer, device=0)

In [23]:
a = poem('<s> mùa xuân')
print(a[0]['generated_text'])

<s> mùa xuân ai cũng ước ao 
 tiên tranh tổ lý đại dương 
 
 đất này của của ta ăn tươi 
 cần chi em đã thật trần ta 
 cần ta như trái ngọt ngào đất trời 
 việt tiên tổ muôn nơi ta 
 càng buồn gửi nghĩa văn chương ngày xưa 
 
 chị đi nhiệm nước mây che 
 trái vườn chuối trúc lung linh động non 
 mẹ ta luôn phải qua mau 
 làng ta gom gió trầm đinh hiền 
 còn ta như thể ngu đất trời 
 trái xanh ngọc bụi mắc đường cuộc đời 
 
 con tim đợi người dưng 
 lòng ta tha thiết tha ánh cờ cuối chiều 
 mẹ bà bảo của ta 
 tự hào ra cố quốc bao giờ là buồn 
 trăng nhìn em đã thành gặp 
 dâng lên nặng kiếp người hào thị phi 
 
 lòng ta tặng ngủ trong tim 
 bởi vì dịu dàng bè say phố phường 
 
 mai xưa sớm tối tìm trầm 
 ông xưa tôi thấp trời để nên 
 đợi bầy dành đợi lỡ làng chung quanh 
 còn ta giờ ở gặp người 
 bằng khi lấy biết bây giờ gặp nhau 
 biết còn lại hãy còn duyên cười 
 
 chiếc nón khăn sợi đồng gần 
 chuyện chu đáo ra sự đời phải bì 
 lấy bầy cò trắng buông neo 
 thiếu mầm chồi biếc chưa

In [24]:
b = poem('<s> anh')
print(b[0]['generated_text'])

<s> anh hùng suốt đời 
 
 lưng trâu cả đất quê quên 
 núi đồi sim sắc màu thề thề quân 
 giếng cồn cát trắng mái tranh 
 trải đôi mắt lắm thương thương bây giờ 
 ban mai mỏi cánh vương vấn vương 
 con chim nặng hạt sương lờ trôi 
 lòng như cánh bướm nhẹ từng cơn 
 lam cành khởi sắc bên cồn cào 
 người đi hơi ấm áp tình thơ 
 binh tinh sương trắng lối triền miên 
 biết đâu mở ý ngơ ngẩn ngơ 
 thương nhung nhớ dấu nụ cười thôi 
 mây trôi nổi cánh diều cuối chiều 
 câu thơ rớt giọt sương ngà chiêm bao 
 
 phố phường tông chợ quê hương 
 chút tà áo bềnh bồng bềnh bồng bềnh 
 chiến hào quang vị tênh loang 
 câu thơ tiếng ếch tháng ngày qua 
 
 gửi vào trong cõi phù sa sa 
 vấn vương vương vấn vít bao tình 
 bóng ai thả một nụ cười dáng tiên 
 hội làng một thoáng quan san 
 làng tôi gửi gắm mấy trang đài tiên 
 thả nghiêng đôi mát lành cao 
 mái chèo khua nhẹ cánh hoa đại ngàn 
 nắng vương vương nợ tiếng trầm ngâm 
 hòa ngát góc mo trước núi cao 
 mâm cơm vạt nắng hanh hao 
 lỡ chân hạ nhắn 